# Imports

In [2]:
import sys
sys.path.append('/Users/rileyoest/VS_Code/HoofStudy')
from src.visualize import *
from src.utils import *
from src.modeling import *

# Get and Preprocess Data

In [2]:
df = get_data()
df, normalization_params = preprocess_data(df)

# Train-Val-Test Split

In [3]:
input_columns = ['DW_y', 'DW_z', 'SM_y', 'SM_z', 'CB_y', 'CB_z']
target_columns = ['P3_y', 'P3_z']
X_train, y_train, X_val, y_val, X_test, y_test, trial_lengths, health_shoe_params = train_test_split(
    dataframe=df,
    input_columns=input_columns,
    target_columns=target_columns,
    train_ratio=0.65,
    val_ratio=0.25,
    seed=0,
    verbose=True
)

Train set:
  (True, 'Eggbar'): 13 trials
  (True, 'Unshod'): 10 trials
  (False, 'Heartbar'): 15 trials
  (True, 'Standard'): 11 trials
  (False, 'Unshod'): 15 trials
  (False, 'Standard'): 15 trials
  (False, 'Eggbar'): 18 trials
  (True, 'Heartbar'): 14 trials
Validate set:
  (True, 'Eggbar'): 5 trials
  (True, 'Unshod'): 4 trials
  (False, 'Heartbar'): 6 trials
  (True, 'Standard'): 4 trials
  (False, 'Unshod'): 6 trials
  (False, 'Standard'): 6 trials
  (False, 'Eggbar'): 7 trials
  (True, 'Heartbar'): 5 trials
Test set:
  (True, 'Eggbar'): 3 trials
  (True, 'Unshod'): 2 trials
  (False, 'Heartbar'): 3 trials
  (True, 'Standard'): 2 trials
  (False, 'Unshod'): 3 trials
  (False, 'Standard'): 3 trials
  (False, 'Eggbar'): 4 trials
  (True, 'Heartbar'): 4 trials


# Training Singular Model

In [6]:
model = NeuralNet(X_train.shape[1], y_train.shape[1], hidden_neurons=32)
model = torch.nn.DataParallel(model)
best_model = train_neural_network(
    model, X_train, y_train, X_val, y_val, 
    num_epochs=250,
    learning_rate=0.01,
    verbose=True, 
    )

Epoch 1/250 - mse train loss: 0.2314 - mse val loss: 0.1092
Epoch 2/250 - mse train loss: 0.1073 - mse val loss: 0.0332
Epoch 3/250 - mse train loss: 0.0324 - mse val loss: 0.0290
Epoch 4/250 - mse train loss: 0.0292 - mse val loss: 0.0638
Epoch 5/250 - mse train loss: 0.0644 - mse val loss: 0.0560
Epoch 6/250 - mse train loss: 0.0567 - mse val loss: 0.0287
Epoch 7/250 - mse train loss: 0.0291 - mse val loss: 0.0125
Epoch 8/250 - mse train loss: 0.0125 - mse val loss: 0.0129
Epoch 9/250 - mse train loss: 0.0126 - mse val loss: 0.0205
Epoch 10/250 - mse train loss: 0.0198 - mse val loss: 0.0251
Epoch 11/250 - mse train loss: 0.0242 - mse val loss: 0.0236
Epoch 12/250 - mse train loss: 0.0227 - mse val loss: 0.0182
Epoch 13/250 - mse train loss: 0.0173 - mse val loss: 0.0128
Epoch 14/250 - mse train loss: 0.0119 - mse val loss: 0.0100
Epoch 15/250 - mse train loss: 0.0092 - mse val loss: 0.0095
Epoch 16/250 - mse train loss: 0.0088 - mse val loss: 0.0088
Epoch 17/250 - mse train loss: 0.

# Save Model

In [10]:
save_model(best_model, 'best_model.pth')

# Load Model

In [9]:
# model = load_model(X_train.shape[1], y_train.shape[1], 'best_model.pth')
model = load_model(len(input_columns), len(target_columns), 'best_model.pth')

In [10]:
print_model_accuracy(model, X_test, y_test)

Accuracy of Predictions for P3 Model
R² = 0.9818
MSE = 0.0012


# Bayesian Optimization to Find Best Hyperparameters

In [14]:
run_bayesian_optimization(get_data, preprocess_data)

|   iter    |  target   | hidden... | learni... | num_ep... | remain... | train_... |
-------------------------------------------------------------------------------------
Parameters:
Train ratio = 0.4587
Val ratio = 0.1334
epochs = 200
Learning rate = 0.07204
Hidden neurons = 32

MSE = 0.001376
| 1         | -0.001376 | 5.668     | 0.07204   | 200.0     | 0.3023    | 0.4587    |
Parameters:
Train ratio = 0.6155
Val ratio = 0.1129
epochs = 234
Learning rate = 0.01863
Hidden neurons = 16

MSE = 0.00128
| 2         | -0.00128  | 4.369     | 0.01863   | 234.6     | 0.3968    | 0.6155    |
Parameters:
Train ratio = 0.5227
Val ratio = 0.282
epochs = 235
Learning rate = 0.03695
Hidden neurons = 32

MSE = 0.00127
| 3         | -0.00127  | 5.438     | 0.03695   | 235.2     | 0.7476    | 0.5227    |
Parameters:
Train ratio = 0.5674
Val ratio = 0.3175
epochs = 245
Learning rate = 0.09667
Hidden neurons = 128

MSE = 0.002109
| 4         | -0.002109 | 7.94      | 0.09667   | 245.5     | 0.9544    

# Cross Validation

In [3]:
df = get_data()
df, _ = preprocess_data(df)

Loading dataframe from cache
Loading preprocessed dataframe from cache


In [6]:
input_columns = ['DW_y', 'DW_z', 'SM_y', 'SM_z', 'CB_y', 'CB_z']
target_columns = ['P3_y', 'P3_z']

In [8]:
cross_validation(df, input_columns, target_columns)

Processing fold 1...
Fold 1: R² = 0.9783, MSE = 0.0015
Processing fold 2...
Fold 2: R² = 0.9819, MSE = 0.0012
Processing fold 3...
Fold 3: R² = 0.9802, MSE = 0.0014
Processing fold 4...
Fold 4: R² = 0.9835, MSE = 0.0012
Processing fold 5...
Fold 5: R² = 0.9797, MSE = 0.0014

Average R²: 0.9807, Average MSE: 0.0013


# Visualization

In [17]:
input_columns = ['DW_y', 'DW_z', 'SM_y', 'SM_z', 'CB_y', 'CB_z']
target_columns = ['P3_y', 'P3_z']

plot_model(NeuralNet, input_features=input_columns, output_features=target_columns, hidden_neurons=32)

In [12]:

df, norm_params = preprocess_data(get_data())
input_columns = ['DW_y', 'DW_z', 'SM_y', 'SM_z', 'CB_y', 'CB_z']
target_columns = ['P3_y', 'P3_z']
X_train, y_train, X_val, y_val, X_test, y_test, trial_lengths, health_shoe_params = train_test_split(
    dataframe=df,
    input_columns=input_columns,
    target_columns=target_columns,
    train_ratio=0.65,
    val_ratio=0.25,
    seed=0,
    verbose=False
)
model = load_model(len(input_columns), len(target_columns), 'best_model.pth')
plot_model_results_to_pdf(model, X_test, y_test, health_shoe_params, trial_lengths, norm_params, get_confidence_intervals, resample_array)

Accuracy of Predictions for P3 Model
R² = 0.9818
MSE = 0.0012
